# Код для статьи по экстрактивой суммаризации

Работа выполняетсч для ODS дата-феста  

## Подходы к суммаризации
---
1. `sumy` - самая последняя библиотека со всеми алгоритмами 
2. `summa` - стандартный пакет без приколов
3. `spacy` - для скачивани языковых моделей, но и обработка
4. `nltk` - 


## Типичные вакансии для Работа.ру
---

1.   История работодателя занимает большую часть, по сути мало - **проблема в экстрактии сути**
2.   Очень короткий однострочник - **невозможность ранжирования предложения в вакансии**
3.   Достаточная вакансия, все по пунктам - **проблема в точности экстракции**


## Реализация библиотекой Sumy
---

*   LexRank
*   Luhn
*   LSA
*   TextRank
*   Edmundson - настроить мб что-то выйдет
*   KL Divergence

## Загрузка библиотек

In [1]:
# !pip uninstall gensim -y

In [2]:
# !pip install sumy spacy nltk pandas
# !pip install summa razdel gensim==3.7.3

## Мой тестовый датасет

In [3]:
# @title Датасет

# text_1 - https://spb.rabota.ru/vacancy/47716438/?search_id=1683195489792cc791p5nz25
# text_1 - https://spb.rabota.ru/vacancy/47712789/?search_id=1683195875196yx8r3gz509j
# text_3 - https://spb.rabota.ru/vacancy/47716362/?search_id=1683195489792cc791p5nz25

text_1 = """
Сбер — это 14 тысяч подразделений по всей стране, а
масштабы компании и система поддержки сотрудников открывают возможности для
карьеры в любом направлении.
 
Присоединяйся к команде малого бизнеса Сбера! На позиции «Менеджера по работе с ключевыми клиентами» ты будешь общаться с предпринимателями и руководителями компаний малого бизнеса и предлагать лучшие решения от Сбера.
 
Эта вакансия для энергичных и общительных людей, готовых вникать в тонкости бизнеса наших
клиентов. Мы поможем тебе освоить специальность, сформируем для тебя базу
клиентов, предоставим возможности карьерного роста.
 
Эта работа подойдет тебе, если ты:
 
·     специалист с высшим или неполным высшим образованием
·     имеешь опыт работы в сфере продаж корпоративным клиентам
·     понимаешь основы ведения бизнеса
·     умеешь выстраивать отношения с людьми
·     готов к разъездной работе.
 
 Тебе предстоит:
 
·     развивать бизнес клиентов из закрепленной базы, предлагая продукты и
услуги Сбера
·     проводить переговоры с клиентами на территории компаний-партнеров
·     привлекать новых клиентов на обслуживание в Сбер
·     проводить переговоры с предпринимателями и руководителями компаний
·     выстраивать долгосрочное сотрудничество с клиентами.
 
Работа в Сбере – это:
 
·     стабильный оклад и социальная поддержка сотрудников
·     официальное оформление с первого дня
·     мобильная связь, корпоративный ipad и оплата транспортных расходов
·     корпоративное обучение в Виртуальной школе Сбера
·     расширенный ДМС с первого дня и льготное страхование для близких
·     бесплатная подписка СберПрайм+, скидки на продукты компаний-партнеров
·     корпоративная пенсионная программа
·     ипотека выгоднее на 4% для каждого сотрудника.
 
 
Присоединяйся к команде менеджеров по работе с ключевыми клиентами малого бизнеса Сбера!"""

text_2 = """
Требования:
по ремонту и обслуживанию электрооборудования 4/5 разрядов. График работы 5-дневка, пн-чт с 8.00 до 17.00, пт с 8.00 до 15.20. Белая з/п. Оформление по ТК РФ. Проход на территорию завода по предварительной записи. З/п от 60 т.р.
Информация о работодателе: Средне-Невский судостроительный завод."""

text_3 = """
Обязанности: 

организация процесса проектирования объектов капитального строительства, капитального ремонта, реконструкции (проектная, прохождение экспертизы, рабочая документация, авторский надзор)
составление технических заданий (ТЗ) на разработку проектной документации и заданий на инженерные изыскания
личная проверка документации
проведение совещаний в процессе проектирования (с заказчиком, с исполнителями)
работа со специалистами в штате и физическими лицами на подряде
постановка задач и руководство коллективом проектировщиков в ходе выполнения проекта от сбора исходных данных до сдачи готового проекта стадии рабочий проект
сбор исходных данных, запрос ТУ, проведение необходимых согласований и взаимодействие с муниципальными органами и службами города
обеспечение соответствия решений предпроектной и утвержденной проектной документации – заданию на проектирование и рабочей документации
взаимодействие с заказчиками


Требования: 

понимание последовательности и взаимосвязи разделов документации между собой
умение читать и понимать каждый раздел документации
умение работать в Автокаде на уровне уверенного пользователя
желание получения новых знаний и развития в профессии
опыт работы с проектами капитального ремонта будет преимуществом


Условия: 

оформление по ТК РФ
график работы 5/2 с 9 до 18
Офис расположен у м. Бухарестская;
Стабильная з/п
размер заработной платы уточняется по результатам собеседования"""

## Функции для очистки датасета

In [10]:
# @title Предварительная чистка датасета

import re
import spacy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
tok = nltk.data.load('tokenizers/punkt/russian.pickle')

stop_words = nltk.corpus.stopwords.words('russian')

def cleaning_text(text):
#     text = text.lower() # Convert to lowercase
    text = text.strip()
    text = re.sub('<[^>]+>', '',text ,flags = re.DOTALL)
    text = re.sub(r"\n", ". ", text)  # Replace ( · ) 
    text = re.sub(r"\s·", ". ", text)  # Replace ( · )
    text = re.sub(r"\W\W\W", ". ", text)  # Replace ( · )
    text = re.sub(r"\W\W", ". ", text)  # Replace ( · ) 
    text = re.sub("/[^\.\,\-\_\'\"\@\?\!\:\$ a-zA-Z0-9А-Яа-я()]/u" ,"", text)
    text = re.sub('\s\W', ' ', text)
    text = re.sub('Требования.', '', text)
    text = re.sub('Обязанности.', '', text)
    text = re.sub('Требуемые навыки:', '', text)
    text = re.sub('\W,\s', '', text) # (Любая не-буква, не-цифра и не подчёркивание) (Любой пробельный символ)
    text = re.sub('\s+', ' ', text) # Любой пробельный символ
#     text = ' '.join([i for i in text.split() if i not in stop_words]) # Remove stop_words
    text = re.sub("nbsp", "", text)
    text = re.sub('\W\s\W', '.', text)
    text = re.sub('{', '', text)
    text = re.sub('}', '', text)
#     print("::::: Text_Cleaned :::::")
    return text

def recleaning_text(text):
#     text = text.lower() # Convert to lowercase
    text = re.sub('<[^>]+>', '',text ,flags = re.DOTALL)
    text = re.sub(r"\n\n", "\n", text)  # Replace ( · )
    text = re.sub(r"\n", ". ", text)  # Replace ( · ) 
    text = re.sub(r"\s·", ". ", text)  # Replace ( · )
    text = re.sub(r"\W\W\W", ". ", text)  # Replace ( · )
    text = re.sub(r"\W\W", ". ", text)  # Replace ( · ) 
    text = re.sub("/[^\.\,\-\_\'\"\@\?\!\:\$ a-zA-Z0-9А-Яа-я()]/u" ," ", text)
    text = re.sub('\s\W', ' ', text)
    text = re.sub('\W,\s', ' ', text) # (Любая не-буква, не-цифра и не подчёркивание) (Любой пробельный символ)
    text = re.sub('\s+', ' ', text) # Любой пробельный символ
    text = re.sub('{', '', text)
    text = re.sub('}', '', text)
    print("::::: Text_Cleaned :::::")
    return text[:200]

# text = recleaning_text(text_1)
# print(text)
# print(len(text))
print(cleaning_text(text_3))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'text_3' is not defined

## Реализация через sumy 

In [15]:
# @title Реализация через `sumy`
import nltk
import pandas as pd
from string import punctuation
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from nltk.corpus import stopwords

nltk.download('punkt')
russian_stopwords = stopwords.words("russian")


def sumy_LexRank(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")    
    summarizerLexRank = LexRankSummarizer(stemmer)
    summary = ''
    for sentenceLexRank in summarizerLexRank(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLexRank)
    return summary

def sumy_LexRank_no_stemmer(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    summarizerLexRank = LexRankSummarizer()
    summary = ''
    for sentenceLexRank in summarizerLexRank(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLexRank)
    return summary

def sumy_LexRank_eng(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    stemmer = Stemmer("russian")
    summarizerLexRank = LexRankSummarizer(stemmer)
    summary = ''
    for sentenceLexRank in summarizerLexRank(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLexRank)
    return summary


# def sumy_LexRank2(text, sentences=3):
#     parser = PlaintextParser.from_string(text, Tokenizer("russian"))
#     stemmer = Stemmer("russian")
#     summarizerLexRank = LexRankSummarizer(stemmer)
#     summary = summarizerLexRank(parser.document,sentences)
#     return summary

def sumy_Luhn(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")
    summarizerLuhn = LuhnSummarizer(stemmer)
    summary = ''
    for sentenceLuhn in summarizerLuhn(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLuhn)
    return summary

def sumy_LSA(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")
    summarizerLSA = LsaSummarizer(stemmer)
    summary = ''
    for sentenceLSA in summarizerLSA(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLSA)
    return summary

def sumy_LSA_nostem(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizerLSA = LsaSummarizer()
    summary = ''
    for sentenceLSA in summarizerLSA(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceLSA)
    return summary

def sumy_TextRank(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")
    summarizerTR = TextRankSummarizer(stemmer)
    summary = ''
    for sentenceTextRank in summarizerTR(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceTextRank)
    return summary

def sumy_KLdiv(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    stemmer = Stemmer("russian")
    summarizerKL = KLSummarizer(stemmer)
    summary = ''
    for sentenceKL in summarizerKL(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceKL)
    return summary

def sumy_KLdiv_no_stem(text, sentences=3):
    parser = PlaintextParser.from_string(text, Tokenizer("russian"))
    summarizerKL = KLSummarizer()
    summary = ''
    for sentenceKL in summarizerKL(parser.document, sentences_count=sentences):
        summary+=' ' + str(sentenceKL)
    return summary


# print("\n====== Edmundson ======")
# summarizerEdmundson = EdmundsonSummarizer(stemmer)

# summarizerEdmundson.bonus_words = ("this", "is", "I", "am", "and",)
# summarizerEdmundson.null_words = ("word", "another", "and", "some", "next",)
# summarizerEdmundson.stigma_words = ("stigma",)
# # summarizerEd.bonus_words = ('focus', 'proposed', 'method', 'describes')
# # summarizerEd.stigma_words = ('example')
# # summarizerEd.null_words = ('literature', 'however')

# for sentenceEdmundson in summarizerEdmundson(parser.document, SENTENCES_COUNT):
#     print(sentenceEdmundson)


# pd.options.display.max_colwidth = 200
# df = pd.DataFrame([['Luhn', SENTENCES_COUNT, text_1, ItogTR, 'count'],
#                   ['TextRank',SENTENCES_COUNT, ItogTR, 'count'],
#                   ['LSA',SENTENCES_COUNT, ItogTR, 'count']], 
#                   columns=['Summarizer','sentense_count', 'normtext',
#                            'sumtext','type_length']
#                   )
# print(df.head)
# df
# text = cleaning_text(text_1)
# print(text)
# sentence = 3

# print("\n====== LexRank ======")
# print(sumy_LexRank(text, sentence))

# print("\n====== reclean LexRank ======")
# print(recleaning_text(sumy_LexRank(text, sentence)))

# # print("\n====== LexRank no stemmer ======")
# print(sumy_LexRank_no_stemmer(text, sentence))

# print("\n====== LexRank eng ======")
# print(sumy_LexRank_eng(text, sentence))

# print("\n====== Luhn ======")
# print(sumy_Luhn(text, sentence))

# print("\n====== TextRank ======")
# print(sumy_TextRank(text, sentence))

# print("\n====== LSA ======")
# print(sumy_LSA(text, sentence))

# print("\n====== KLdiv ======")
# print(sumy_KLdiv(text, sentence))

# print("\n====== KLdiv_no stem ======")
# print(sumy_KLdiv_no_stem(text, sentence))

# print("\n====== LSA no stem ======")
# print(sumy_LSA_nostem(text, sentence))

# print("\n====== KL Divergence ======")
# print(sumy_KLdiv(text, sentence)[:200])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Реализация через `summa`

In [6]:
from summa import summarizer
from summa.summarizer import summarize

# text = cleaning_text(text_1)
# summarize = summarize_summa_words(text, words=40)
# summarize = recleaning_text(summarize)

def summa_summarize_ratio(text, ratio=0.1):
    summarization = summarize(text, ratio=ratio, language='russian')
    return summarization

def summa_summarize_words(text, words=30):
    summarization = summarize(text, words=words,  language='russian')
    return summarization

## Реализация через Spacy

## Реализация через Gensim (bm25)

In [7]:
from gensim.summarization.summarizer import summarize
def gensim_summarizer(text, word_count=30):
    summarization = summarize(text, word_count=word_count)
    return summarization

text = text_1
print(gensim_summarizer(text))

На позиции «Менеджера по работе с ключевыми клиентами» ты будешь общаться с предпринимателями и руководителями компаний малого бизнеса и предлагать лучшие решения от Сбера.
Присоединяйся к команде менеджеров по работе с ключевыми клиентами малого бизнеса Сбера!


## tf-idf NLTK самописный

In [8]:

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer
import math

ps = PorterStemmer()


def sent_preprocessing(sentences: list) -> list:
    cleaned_sentencs = [sent for sent in sentences if sent]
    for sent in sentences:
        if sent == "" or sent == " ":
            print(1)
    return cleaned_sentencs


def text_preprocessing(sentences: list):
    """
    Pre processing text to remove unnecessary words.
    """
    # print('Preprocessing text')

    stop_words = set(stopwords.words("english"))

    clean_words = None
    for sent in sentences:
        words = word_tokenize(sent)
        words = [ps.stem(word.lower()) for word in words if word.isalnum()]
        clean_words = [word for word in words if word not in stop_words]

    return clean_words


def create_tf_matrix(sentences: list) -> dict:
    """
    Here document refers to a sentence.
    TF(t) = (Number of times the term t appears in a document) / (Total number of terms in the document)
    """
    # print('Creating tf matrix.')

    tf_matrix = {}

    for sentence in sentences:
        tf_table = {}

        clean_words = text_preprocessing([sentence])
        words_count = len(word_tokenize(sentence))

        # Determining frequency of words in the sentence
        word_freq = {}
        for word in clean_words:
            word_freq[word] = (word_freq[word] + 1) if word in word_freq else 1

        # Calculating relative tf of the words in the sentence
        for word, count in word_freq.items():
            tf_table[word] = count / words_count

        tf_matrix[sentence[:15]] = tf_table

    return tf_matrix


def create_idf_matrix(sentences: list) -> dict:
    """
    Inverse Document Frequency.
    IDF(t) = log_e(Total number of documents / Number of documents with term t in it)
    """
    # print('Creating idf matrix.')

    idf_matrix = {}
    documents_count = len(sentences)
    sentence_word_table = {}

    # Getting words in the sentence
    for sentence in sentences:
        clean_words = text_preprocessing([sentence])
        sentence_word_table[sentence[:15]] = clean_words

    # Determining word count table with the count of sentences which contains the word.
    word_in_docs = {}
    for sent, words in sentence_word_table.items():
        for word in words:
            word_in_docs[word] = (word_in_docs[word] + 1) if word in word_in_docs else 1

    # Determining idf of the words in the sentence.
    for sent, words in sentence_word_table.items():
        idf_table = {}
        for word in words:
            idf_table[word] = math.log10(documents_count / float(word_in_docs[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix


def create_tf_idf_matrix(tf_matrix, idf_matrix) -> dict:
    """
    Create a tf-idf matrix which is multiplication of tf * idf individual words
    """
    # print('Calculating tf-idf of sentences.')

    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(
        tf_matrix.items(), idf_matrix.items()
    ):
        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(), f_table2.items()):
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix


def create_sentence_score_table(tf_idf_matrix) -> dict:
    """
    Determining average score of words of the sentence with its words tf-idf value.
    """
    # print('Creating sentence score table.')

    sentence_value = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0
        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        smoothing = 1
        sentence_value[sent] = (total_score_per_sentence + smoothing) / (
            count_words_in_sentence + smoothing
        )

    return sentence_value


def find_average_score(sentence_value):
    """
    Calculate average value of a sentence form the sentence score table.
    """
    # print('Finding average score')

    sum = 0
    for val in sentence_value:
        sum += sentence_value[val]

    average = sum / len(sentence_value)

    return average


def generate_summary(sentences, sentence_value, threshold):
    """
    Generate a sentence for sentence score greater than average.
    """
    # print('Generating summary')

    sentence_count = 0
    summary = ""

    for sentence in sentences:
        if (
            sentence[:15] in sentence_value
            and sentence_value[sentence[:15]] >= threshold
        ):
            summary += sentence + " "
            sentence_count += 1

    return summary


def tfidf_sum_pipeline(text):

    sentences = sent_tokenize(text)
    print("Sentences", sentences)

    # sentences = sent_preprocessing(sentences)

    tf_matrix = create_tf_matrix(sentences)
    # print('TF matrix', tf_matrix)

    idf_matrix = create_idf_matrix(sentences)
    # print('IDF matrix',idf_matrix)

    tf_idf_matrix = create_tf_idf_matrix(tf_matrix, idf_matrix)
    # print('TF-IDF matrix', tf_idf_matrix)
    # print('First document tfidf',tf_idf_matrix[list(tf_idf_matrix.keys())[0]])

    sentence_value = create_sentence_score_table(tf_idf_matrix)
    # print('Sentence Scores', sentence_value)

    threshold = find_average_score(sentence_value)
    # print('Threshold', threshold)

    summary = generate_summary(sentences, sentence_value, threshold)

    return summary


#     # print('\nOriginal document\n',text,end='\n'*2)
#     print('Summary\n', summary)

#     print()
#     print(f'Original {len(sent_tokenize(text))} sentences, Summarized {len(sent_tokenize(summary))} sentences')

In [9]:
!spacy download ru_core_news_sm
import spacy
from spacy.lang.ru.stop_words import STOP_WORDS
from string import punctuation
from heapq import nlargest

def spacy_summarize_ratio(text, per):
    nlp = spacy.load('ru_core_news_sm')
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={}
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency
    sentence_tokens= [sent for sent in doc.sents]
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():                            
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()]
    select_length=int(len(sentence_tokens)*per)
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
    final_summary=[word.text for word in summary]
    summary=''.join(final_summary)
    return summary
text = cleaning_text(text_1)
spacy_summarize_ratio(text, 0.1)

2023-05-19 08:56:43.237220: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
^C
Traceback (most recent call last):
  File "/opt/conda/bin/spacy", line 5, in <module>
    from spacy.cli import app
  File "/opt/conda/lib/python3.8/site-packages/spacy/__init__.py", line 10, in <module>
    from thinc.api import prefer_gpu, require_gpu, require_cpu  # noqa: F401
  File "/opt/conda/lib/python3.8/site-packages/thinc/api.py", line 2, in <module>
    from .initializers import normal_init, uniform_init, glorot_uniform_init, zero_init
  File "/opt/conda/lib/python3.8/site-packages/thinc/initializers.py", line 4, in <module>
    from .backends import Ops
  File "/opt/conda/lib/python3.8/site-packages/thinc/backends/__init__.py", line 7, in <module>
    from .ops import Ops
  File "/opt/conda/lib/python3.8/site-packages/thinc/backends/ops.py", line 13, in <module>
    from ..util import get_array_module, is_xp_array, to_numpy
  F

OSError: [E050] Can't find model 'ru_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
%time
tfidf_sum_pipeline(cleaning_text(text_1))

## tf-idf через sklearn

In [ ]:
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import pandas as pd


def create_sentence_score_table(sentences, tfidf_vector) -> dict:
    """
    Determining average score of words of the sentence with its words tf-idf value.
    """
    # print('Creating sentence score table.')

    sentence_value = {}

    for sentence, sent_vector in zip(sentences, tfidf_vector):
        df = pd.DataFrame(sent_vector.T.todense(), columns=['tfidf'])
        total_score_per_sentence = df.sum()['tfidf']
        count_words_in_sentence = len(sentences)
        smoothing = 1
        sentence_value[sentence[:15]] = (total_score_per_sentence + smoothing) / (count_words_in_sentence + smoothing)

    return sentence_value


def find_average_score(sentence_value):
    """
    Calculate average value of a sentence form the sentence score table.
    """
    # print('Finding average score')

    sum = 0
    for val in sentence_value:
        sum += sentence_value[val]

    average = sum / len(sentence_value)

    return average


def generate_summary(sentences, sentence_value, threshold):
    """
    Generate a sentence for sentence score greater than average.
    """
    # print('Generating summary')

    sentence_count = 3
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentence_value and sentence_value[sentence[:15]] >= threshold:
            summary += sentence + " "
            sentence_count += 1

    return summary


def tfidf_sum_nltk_pipeline(text):

    sentences = sent_tokenize(text)  # docs
    # print('Sentences', sentences)

    cv = CountVectorizer()
    word_count_vector = cv.fit_transform(sentences)
    # print('Word count vector', word_count_vector.shape, word_count_vector)

    feature_names = cv.get_feature_names()
    # print('Feature names',len(feature_names),feature_names)

    tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
    tfidf_transformer.fit(word_count_vector)

    df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(), columns=['idf_weights'])
    df_idf.sort_values(by=['idf_weights'])
    # print(df_idf.head())
    # print(df_idf.tail())

    count_vector = cv.transform(sentences)
    tfidf_vector = tfidf_transformer.transform(count_vector)

    first_document_vector = tfidf_vector[0]
    df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=['tfidf'])
    df.sort_values(by=['tfidf'], ascending=False)

    sentence_value = create_sentence_score_table(sentences, tfidf_vector)
    # print('Sentence Scores', sentence_value)

    threshold = find_average_score(sentence_value)
    # print('Threshold', threshold)

    summary = generate_summary(sentences, sentence_value, threshold)

    return summary


In [ ]:
%time
tfidf_sum_nltk_pipeline(text)

## Самописный ТекстРанк

In [ ]:
import networkx as nx
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.metrics.pairwise import cosine_similarity

wl = WordNetLemmatizer()


def extract_word_vectors() -> dict:
    """
    Extracting word embeddings. These are the n vector representation of words.
    """
    print('Extracting word vectors')

    word_embeddings = {}
    # Here we use glove word embeddings of 100 dimension
    f = open('../../Files/glove.6B/glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs

    f.close()
    return word_embeddings


def text_preprocessing(sentences: list) -> list:
    """
    Pre processing text to remove unnecessary words.
    """
    print('Preprocessing text')

    stop_words = set(stopwords.words('russian'))

    clean_words = None
    for sent in sentences:
        words = word_tokenize(sent)
        words = [wl.lemmatize(word.lower()) for word in words if word.isalnum()]
        clean_words = [word for word in words if word not in stop_words]

    return clean_words


def sentence_vector_representation(sentences: list, word_embeddings: dict) -> list:
    """
    Creating sentence vectors from word embeddings.
    """
    print('Sentence embedding vector representations')

    sentence_vectors = []
    for sent in sentences:
        clean_words = text_preprocessing([sent])
        # Averaging the sum of word embeddings of the sentence to get sentence embedding vector
        v = sum([word_embeddings.get(word, np.zeros(100, )) for word in clean_words]) / (len(clean_words) + 0.001)
        sentence_vectors.append(v)

    return sentence_vectors


def create_similarity_matrix(sentences: list, sentence_vectors: list) -> np.ndarray:
    """
    Using cosine similarity, generate similarity matrix.
    """
    print('Creating similarity matrix')

    # Defining a zero matrix of dimension n * n
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                # Replacing array value with similarity value.
                # Not replacing the diagonal values because it represents similarity with its own sentence.
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1, 100), sentence_vectors[j].reshape(1, 100))[0, 0]

    return sim_mat


def determine_sentence_rank(sentences: list, sim_mat: np.ndarray):
    """
    Determining sentence rank using Page Rank algorithm.
    """
    print('Determining sentence ranks')
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    ranked_sentences = sorted([(scores[i], s[:15]) for i, s in enumerate(sentences)], reverse=True)
    return ranked_sentences


def generate_summary(sentences: list, ranked_sentences: list):
    """
    Generate a sentence for sentence score greater than average.
    """
    print('Generating summary')

    # Get top 1/3 th ranked sentences
    top_ranked_sentences = ranked_sentences[:int(len(sentences) / 3)] if len(sentences) >= 3 else ranked_sentences

    sentence_count = 0
    summary = ''

    for i in sentences:
        for j in top_ranked_sentences:
            if i[:15] == j[1]:
                summary += i + ' '
                sentence_count += 1
                break

    return summary


def textrank_pipeline(text):

    sentences = sent_tokenize(text)
    print('Sentences',sentences)

    word_embeddings = extract_word_vectors()
    print('Word embeddings',len(word_embeddings))

    sentence_vectors = sentence_vector_representation(sentences, word_embeddings)
    print('Sentence vectors', len(sentence_vectors), sentence_vectors)

    sim_mat = create_similarity_matrix(sentences, sentence_vectors)
    print('Similarity matrix', sim_mat.shape, sim_mat)

    ranked_sentences = determine_sentence_rank(sentences, sim_mat)
    print('Ranked sentences', ranked_sentences)

    summary = generate_summary(sentences, ranked_sentences)

    print('\nOriginal document\n',text,end='\n'*2)
    print('Summary\n',summary)


In [ ]:
textrank_pipeline(text_1)

## Обработка датасета

In [1]:
import pandas as pd

df_vacancy_to_summarize = pd.read_csv('summaries_markup.csv')
df_vacancy_to_summarize.head()

,index,vacancy_id,summary_mbart,summary_len_mbart,summary_t5,summary_len_t5,choices,full_text,custom_position,is_paper_vacancy,company_id,person_id,skills,description,summary_human,summary_human_len
0,0,47284448,"Обязанности: Приёмка, расценка, выкладка товар...",176,"Обязанности: Приёмка, расценка, выкладка товар...",201,NaN,"Обязанности: \r\n \r\n Приёмка, расценк...",Продавец-консультант,0,1642393.0,33845359,NaN,<strong><strong>Обязанности:</strong>&nbsp;</s...,NaN,NaN
1,1,47305503,Обязанности: Приемка и контроль качества сырья...,243,"Приемка и контроль качества сырья, участие в п...",202,NaN,Обязанности: \r\n Приемка и контроль качеств...,Оператор станка,0,3739879.0,48936343,NaN,<strong>Обязанности:</strong><br />\r\n<strong...,NaN,NaN
2,2,47306099,В агентство недвижимости требуется кадастровый...,182,Кадастровый инженер В связи с расширением объе...,229,NaN,Кадастровый инженер\r\n\r\nВ связи с расширени...,Кадастровый инженер,0,3738438.0,46000252,NaN,Кадастровый инженер\r\n\r\nВ связи с расширени...,NaN,NaN
3,3,47181790,В крупную российскую марку модной женской одеж...,201,Ольга Гринюк (Olga Grinyuk) - российская марка...,183,NaN,Ольга Гринюк (Olga Grinyuk) - российская марка...,Менеджер по продажам,0,3728185.0,48695724,NaN,Ольга Гринюк (Olga Grinyuk) - российская марка...,NaN,NaN
4,4,47271353,Обязанности: Пошив чехлов на мягкую мебель. Тр...,157,Обязанности: Пошив чехлов на мягкую мебель. Тр...,157,NaN,Обязанности: \r\nПошив чехлов на мягкую м...,Швея-универсал,0,3486999.0,45349740,NaN,<strong><strong>Обязанности:</strong> </strong...,NaN,NaN


In [2]:
df_extract_summarization = df_vacancy_to_summarize[['full_text','description']].copy()
df_extract_summarization.head()

,full_text,description
0,"Обязанности: \r\n \r\n Приёмка, расценк...",<strong><strong>Обязанности:</strong>&nbsp;</s...
1,Обязанности: \r\n Приемка и контроль качеств...,<strong>Обязанности:</strong><br />\r\n<strong...
2,Кадастровый инженер\r\n\r\nВ связи с расширени...,Кадастровый инженер\r\n\r\nВ связи с расширени...
3,Ольга Гринюк (Olga Grinyuk) - российская марка...,Ольга Гринюк (Olga Grinyuk) - российская марка...
4,Обязанности: \r\nПошив чехлов на мягкую м...,<strong><strong>Обязанности:</strong> </strong...


In [3]:
df_vacancy_to_summarize['skills'] = df_vacancy_to_summarize['skills'].fillna(' ')

In [4]:
df_vacancy_to_summarize['skills'] = df_vacancy_to_summarize['skills'].apply(lambda x: 'Требуемые навыки: {}'.format(x) if x!= ' ' else x)
df_vacancy_to_summarize['skills'] = df_vacancy_to_summarize['skills'].apply(lambda x: x.replace('{', ''))
df_vacancy_to_summarize['skills'] = df_vacancy_to_summarize['skills'].apply(lambda x: x.replace('}', ''))

In [5]:
df_extract_summarization['to_summarization'] = df_extract_summarization['full_text'] + '\r\n'+ df_vacancy_to_summarize['skills']

In [6]:
df_extract_summarization['to_summarization'] = df_extract_summarization['to_summarization'].apply(lambda x: str(x).strip())

In [7]:
print(df_extract_summarization["to_summarization"][7])

Предприятие  ООО "Энергомаш"  является разработчиком и изготовителем специальной техники для транспортировки опасных грузов. Продукция предприятия востребована в любом городе и регионе, т.к. выпускаемые нами автоцистерны и полуприцепы-цистерны доставляют ГСМ на городские сети автозаправок, транспортируют нефть в местах нефтедобычи и переработки. Сферы деятельности, для которых предприятие выпускает спецтехнику, очень обширны.  Предприятие на сегодняшний день имеет несколько производственных площадок, обособленных подразделений, чтобы обеспечить возможности комфортной работы для сотрудников.  В связи с расширением производства, мы ищем в нашу команду активного и амбициозного человека, который любит работать и самое главное зарабатывать!      Обязанности:     выполнение работ на токарных станках,  выполнение работ на фрезерных станках,  выполнение работ на сверлильных станках,  Работа на станках класса 1К62, TNTL290 стойка Fanuc  чтение чертежей,  нарезка наружной и внутренней резьбы.   

In [16]:
def summarization_pipline_sumy_LexRank(text):
    text = cleaning_text(text)
    text = sumy_LexRank(text, sentences=3)
#     text = cleaning_text(text)
    return text

def summarization_pipline_sumy_Luhn(text):
    text = cleaning_text(text)
    text = sumy_Luhn(text, sentences=3)
#     text = cleaning_text(text)
    return text

def summarization_pipline_sumy_LSA(text):
    text = cleaning_text(text)
    text = sumy_LSA(text, sentences=3)
#     text = cleaning_text(text)
    return text

def summarization_pipline_sumy_TextRank(text):
    text = cleaning_text(text)
    text = sumy_TextRank(text, sentences=3)
#     text = cleaning_text(text)
    return text

def summarization_pipline_sumy_KLdiv(text):
    text = cleaning_text(text)
    text = sumy_KLdiv(text, sentences=3)
#     text = cleaning_text(text)
    return text

In [17]:
%%time
# resume_count = 300

# df_extract_summarization['sumy_LexRank'] = df_extract_summarization['to_summarization'][:resume_count].apply(summarization_pipline_sumy_LexRank)
# df_extract_summarization['sumy_Luhn'] = df_extract_summarization['to_summarization'][:resume_count].apply(summarization_pipline_sumy_Luhn)
df_extract_summarization['sumy_LSA'] = df_extract_summarization['to_summarization'].apply(summarization_pipline_sumy_LSA)
df_extract_summarization['sumy_TextRank'] = df_extract_summarization['to_summarization'].apply(summarization_pipline_sumy_TextRank)
# df_extract_summarization['sumy_KLdiv'] = df_extract_summarization['to_summarization'][:resume_count].apply(summarization_pipline_sumy_KLdiv)



# df_extract_summarization.head()
# df_extract_summarization[30:35]

/opt/conda/lib/python3.8/site-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (3) is lower than number of sentences (4). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
/opt/conda/lib/python3.8/site-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (9) is lower than number of sentences (10). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
/opt/conda/lib/python3.8/site-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (1) is lower than number of sentences (4). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
/opt/conda/lib/python3.8/site-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (138) is lower than number of sentences (174). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))
/opt/conda/lib/python3.8/site-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words

CPU times: user 1h 52min 12s, sys: 9h 27min 3s, total: 11h 19min 16s
Wall time: 50min 56s


In [42]:
df_extract_summarization[65:70].values[-1]

array(['  Обособленное подразделение ООО "ТрансСтройКомплект" - прямой работодатель! Организация занимается услугами грузового автотранспорта, строительство лежневых дорог!   \r\n  \r\n Обязанности:  \r\n  \r\n \r\n Формирование полной и достоверной информации о порядке формирования данных о величине доходов и расходов организации, определяющих размер налоговой базы отчетного налогового периода, \r\n Осуществление ведения налоговых регистров, расчет налогов, подготовки и сдач налоговой и статистической отчетности, составление реестров начислений и уплаты налогов и сборов в бюджет для сверки с налоговыми органами, прохождение камеральных и выездных проверок \r\n \r\n Требования:   \r\n \r\n необходим опыт работы в транспортных предприятиях, знание организации грузоперевозок \r\n высшее профильное образование \r\n   \r\n \r\n Условия:   \r\n \r\n оформление согласно трудовому законодательству РФ \r\n Пятидневная рабочая неделя с 8-00 до 17-15 \r\n оклад \r\n адрес работодателя: г Тюмень,

In [45]:
print(df_extract_summarization[65:70].sumy_KLdiv.values[-1])

 Формирование полной и достоверной информации о порядке формирования данных о величине доходов и расходов организации. составление реестров начислений и уплаты налогов и сборов в бюджет для сверки с налоговыми органами. прохождение камеральных и выездных проверок.


In [ ]:
sumy_KLdiv

In [79]:
from functools import wraps
import time

def timer(func):
    @wraps(func)
    def wrapper(a, b):
        start_time = time.time();
        retval = func(a, b)
        print("the function ends in ", time.time()-start_time, "secs")
        return retval
    return wrapper

In [47]:
df_extract_summarization

,full_text,description,to_summarization,sumy_LexRank,sumy_Luhn,sumy_LSA,sumy_TextRank,sumy_KLdiv
0,"Обязанности: \r\n \r\n Приёмка, расценк...",<strong><strong>Обязанности:</strong>&nbsp;</s...,"Обязанности: \r\n \r\n Приёмка, расценка ...",Приёмка. расценка. выкладка товара.,Приёмка. расценка. выкладка товара.,выкладка товара. поддержание чистоты. Смена с...,выкладка товара. поддержание чистоты. Смена с...,Приёмка. выкладка товара. Смена с 9:00 до 19:00.
1,Обязанности: \r\n Приемка и контроль качеств...,<strong>Обязанности:</strong><br />\r\n<strong...,Обязанности: \r\n Приемка и контроль качества...,Приемка и контроль качества сырья. участие в ...,Приемка и контроль качества сырья. участие в ...,Приемка и контроль качества сырья. участие в ...,Приемка и контроль качества сырья. участие в ...,Приемка и контроль качества сырья. участие в ...
2,Кадастровый инженер\r\n\r\nВ связи с расширени...,Кадастровый инженер\r\n\r\nВ связи с расширени...,Кадастровый инженер\r\n\r\nВ связи с расширени...,Кадастровый инженер. В связи с расширением об...,Кадастровый инженер. В связи с расширением об...,В связи с расширением объема услуг. в агентст...,в агентство недвижимости требуется кадастровы...,В связи с расширением объема услуг. в агентст...
3,Ольга Гринюк (Olga Grinyuk) - российская марка...,Ольга Гринюк (Olga Grinyuk) - российская марка...,Ольга Гринюк (Olga Grinyuk) - российская марка...,Работа с действующей базой оптовых клиентов. ...,Опыт работы в amoCRM и 1С. Опыт работы в прод...,Вот уже 15 лет все платья создаются в Ростове...,Выпускается 12 коллекций в год. до 80 различн...,На сегодняшний день у нас за плечами более 50...
4,Обязанности: \r\nПошив чехлов на мягкую м...,<strong><strong>Обязанности:</strong> </strong...,Обязанности: \r\nПошив чехлов на мягкую меб...,Пошив чехлов на мягкую мебель. Стаж приветств...,Пошив чехлов на мягкую мебель. Стаж приветств...,Пошив чехлов на мягкую мебель. можно обучение...,Пошив чехлов на мягкую мебель. можно обучение...,Стаж приветствуется. Условия. Рабочий день с ...
...,...,...,...,...,...,...,...,...
49983,"Компания ""Мосэкопак"" приглашает на работу рек...","<p>Компания ""Мосэкопак"" приглашает на работу р...","Компания ""Мосэкопак"" приглашает на работу рекр...",NaN,NaN,NaN,NaN,NaN
49984,Обязанности: \r\n \r\n Поиск новых клиен...,<strong><strong>Обязанности:</strong> </strong...,Обязанности: \r\n \r\n Поиск новых клиенто...,NaN,NaN,NaN,NaN,NaN
49985,Обязанности: \r\n \r\n эксплуатация сетей св...,<strong>Обязанности:</strong><br />\r\n<ul>\r\...,Обязанности: \r\n \r\n эксплуатация сетей свя...,NaN,NaN,NaN,NaN,NaN
49986,Кассир-комплектовщик интернет заказов Компани...,Кассир-комплектовщик интернет заказов<br /> Ко...,Кассир-комплектовщик интернет заказов Компани...,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_extract_summarization.to_csv('extract_summ_final.csv', index=False)

<function go_through_file at 0x7f0d20222160>
